In [1]:
from __future__ import print_function
import os

In [2]:
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
import string

In [3]:
# 파일 읽기
with open('alice_in_wonderland.txt', 'r') as content_file:
    content = content_file.read()

In [4]:
content2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in content]).split())
 
tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

In [5]:
# N-1이 마지막 N번째 단어를 예측하는 데 사용되는 N그램에 대한 N값 선택
N = 3
quads = list(nltk.ngrams(tokens,N))

newl_app = []
for ln in quads:
    newl = " ".join(ln)        
    newl_app.append(newl)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

x_trigm = []
y_trigm = []

for l in newl_app:
    x_str = " ".join(l.split()[0:N-1])
    y_str = l.split()[N-1]   
    x_trigm.append(x_str)
    y_trigm.append(y_str)

x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()

In [7]:
dictnry = vectorizer.vocabulary_
rev_dictnry = {v:k for k,v in dictnry.items()}

X = np.array(x_trigm_check)
Y = np.array(y_trigm_check)

Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X, Y,x_trigm, test_size=0.3,random_state=42)

print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape)
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17947, 2559) Y Train shape (17947, 2559)
X Test shape (7692, 2559) Y Test shape (7692, 2559)


In [8]:
rev_dictnry

{1088: 'in',
 2510: 'wonderland',
 51: 'alice',
 27: 'adventures',
 1242: 'lewis',
 303: 'carroll',
 2205: 'the',
 1363: 'millennium',
 869: 'fulcrum',
 641: 'edition',
 334: 'chapter',
 591: 'down',
 1719: 'rabbit',
 1034: 'hole',
 2430: 'was',
 183: 'beginning',
 2261: 'to',
 895: 'get',
 2398: 'very',
 2258: 'tired',
 1478: 'of',
 1988: 'sitting',
 278: 'by',
 1010: 'her',
 1984: 'sister',
 1491: 'on',
 147: 'bank',
 70: 'and',
 987: 'having',
 1461: 'nothing',
 570: 'do',
 1492: 'once',
 1506: 'or',
 2331: 'twice',
 1925: 'she',
 955: 'had',
 1569: 'peeped',
 1111: 'into',
 225: 'book',
 1743: 'reading',
 272: 'but',
 1124: 'it',
 1448: 'no',
 1591: 'pictures',
 429: 'conversations',
 2456: 'what',
 1122: 'is',
 2378: 'use',
 2228: 'thought',
 2500: 'without',
 428: 'conversation',
 2024: 'so',
 419: 'considering',
 1526: 'own',
 1364: 'mind',
 113: 'as',
 2450: 'well',
 435: 'could',
 835: 'for',
 1047: 'hot',
 506: 'day',
 1307: 'made',
 760: 'feel',
 2001: 'sleepy',
 2130: 'stup

In [9]:
from keras.layers import Input,Dense,Dropout
from keras.models import Model

np.random.seed(42)

BATCH_SIZE = 128
NUM_EPOCHS = 100

input_layer = Input(shape = (Xtrain.shape[1],),name="input")
first_layer = Dense(1000,activation='relu',name = "first")(input_layer)
first_dropout = Dropout(0.5,name="firstdout")(first_layer)

second_layer = Dense(800,activation='relu',name="second")(first_dropout)

third_layer = Dense(1000,activation='relu',name="third")(second_layer)
third_dropout = Dropout(0.5,name="thirdout")(third_layer)

fourth_layer = Dense(Ytrain.shape[1],activation='softmax',name = "fourth")(third_dropout)


history = Model(input_layer,fourth_layer)
history.compile(optimizer = "adam",loss="categorical_crossentropy",metrics=["accuracy"])

Using TensorFlow backend.


In [10]:
history.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS, verbose=1,validation_split = 0.2)

Train on 14357 samples, validate on 3590 samples
Epoch 1/100
14357/14357 [==============================] - 19s 1ms/step - loss: 6.3383 - accuracy: 0.0547 - val_loss: 6.0993 - val_accuracy: 0.0696
Epoch 2/100
14357/14357 [==============================] - 18s 1ms/step - loss: 5.8996 - accuracy: 0.0682 - val_loss: 6.0305 - val_accuracy: 0.0735
Epoch 3/100
14357/14357 [==============================] - 17s 1ms/step - loss: 5.6795 - accuracy: 0.0842 - val_loss: 5.9894 - val_accuracy: 0.0858
Epoch 4/100
14357/14357 [==============================] - 21s 1ms/step - loss: 5.4241 - accuracy: 0.1080 - val_loss: 6.0052 - val_accuracy: 0.0972
Epoch 5/100
14357/14357 [==============================] - 18s 1ms/step - loss: 5.1691 - accuracy: 0.1328 - val_loss: 6.0046 - val_accuracy: 0.1017
Epoch 6/100
14357/14357 [==============================] - 18s 1ms/step - loss: 4.9296 - accuracy: 0.1518 - val_loss: 6.0420 - val_accuracy: 0.1028
Epoch 7/100
14357/14357 [==============================] - 17s 

In [11]:
# 모델 예측
Y_pred = history.predict(Xtest)


# 테스트 데이터에 대한 샘플 확인
print ("Prior bigram words","|Actual","|Predicted","\n")

for i in range(10):
    print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

Prior bigram words |Actual |Predicted 

0 the evening | beautiful | beautiful
1 slipped in | like | the
2 alice swallowing | down | her
3 an encouraging | tone | alice
4 waistcoat pocket | or | repeated
5 she went | on | on
6 that she | knew | was
7 down on | her | one
8 dormouse went | on | on
9 soup soup | of | you


In [12]:
import random
	
	
NUM_DISPLAY = 10
   
for i in random.sample(range(len(xtest_tg)),NUM_DISPLAY):
	print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

5391 alice very | humbly | earnestly
1874 and she | kept | went
7654 said alice | swallowing | very
6951 perfectly round | she | and
6782 much care | where | of
1951 too began | dreaming | to
4452 looked at | alice | it
6104 the house | down | let
6517 to break | the | she
5438 right as | usual | she
